<a href="https://colab.research.google.com/github/jstephens/tideprediction/blob/main/Tide_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries


In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from bokeh.io import show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import DatetimeTickFormatter, DatetimeAxis
from bokeh.models import HoverTool
from bokeh.models import FixedTicker
from bokeh.embed import components
from bokeh.embed import autoload_static
output_notebook()
import imageio
import matplotlib.pyplot as plt
from datetime import datetime
import xgboost as xgb
import datetime
import joblib
from sklearn.multioutput import MultiOutputRegressor

Mounted at /content/drive


# Data Load and Initial Transformation

In [2]:
X_train = np.load('drive/MyDrive/source/X_train_surge_new.npz')
X_test = np.load('drive/MyDrive/source/X_test_surge_new.npz')
Y_train = pd.read_csv('drive/MyDrive/source/Y_train_surge.csv',index_col = 'id_sequence')

df_X_test = pd.DataFrame.from_dict({item: X_test[item] for item in X_test.files}, orient='index').T.set_index('id_sequence')
df_X_train = pd.DataFrame.from_dict({item: X_train[item] for item in X_train.files}, orient='index').T.set_index('id_sequence')

In [3]:
def timestamp_to_datetime(timestamp):
    return pd.to_datetime('1970-01-01') + pd.to_timedelta(timestamp, unit='s')

df_X_train['t_slp'] = df_X_train['t_slp'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_train['t_surge1_input'] = df_X_train['t_surge1_input'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_train['t_surge2_input'] = df_X_train['t_surge2_input'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_train['t_surge1_output'] = df_X_train['t_surge1_output'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_train['t_surge2_output'] = df_X_train['t_surge2_output'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))

# Optional: Input EDA

In [ ]:
df_X_train

,t_slp,slp,t_surge1_input,surge1_input,t_surge2_input,surge2_input,t_surge1_output,t_surge2_output
id_sequence,,,,,,,,
4600,"[2002-03-27 12:00:00, 2002-03-27 15:00:16, 200...","[[[102326.13, 102259.13, 102183.13, 102098.13,...","[2002-03-27 00:00:00, 2002-03-27 12:59:44, 200...","[0.19478741, -0.0029626957, 0.2014908, 0.09088...","[2002-03-27 01:59:28, 2002-03-27 15:00:16, 200...","[-1.0178498, -0.5484413, -0.3640308, 0.6669915...","[2002-04-01 04:00:00, 2002-04-01 16:59:44, 200...","[2002-04-01 06:00:32, 2002-04-01 17:59:28, 200..."
4601,"[2002-03-28 12:00:00, 2002-03-28 15:00:16, 200...","[[[101650.03, 101598.03, 101572.03, 101555.03,...","[2002-03-28 00:59:44, 2002-03-28 14:00:32, 200...","[0.2014908, 0.09088481, -0.023072876, 0.134456...","[2002-03-28 03:00:16, 2002-03-28 15:00:16, 200...","[-0.3640308, 0.6669915, 0.029937062, 0.5831685...","[2002-04-02 04:59:44, 2002-04-02 16:59:44, 200...","[2002-04-02 07:00:16, 2002-04-02 19:00:16, 200..."
4602,"[2002-03-29 12:00:00, 2002-03-29 15:00:16, 200...","[[[101207.48, 101035.48, 100907.48, 100831.48,...","[2002-03-29 01:59:28, 2002-03-29 15:00:16, 200...","[-0.023072876, 0.13445687, 0.054016147, 0.1210...","[2002-03-29 04:00:00, 2002-03-29 16:00:00, 200...","[0.029937062, 0.58316857, 0.24787673, 0.138906...","[2002-04-03 04:59:44, 2002-04-03 16:59:44, 200...","[2002-04-03 07:00:16, 2002-04-03 20:00:00, 200..."
4603,"[2002-03-30 15:00:16, 2002-03-30 17:59:28, 200...","[[[102146.62, 102047.62, 101940.62, 101824.62,...","[2002-03-30 03:00:16, 2002-03-30 15:00:16, 200...","[0.054016147, 0.12105008, 0.04060936, 0.204842...","[2002-03-30 04:00:00, 2002-03-30 16:59:44, 200...","[0.24787673, 0.1389069, 0.3233174, 0.54963934,...","[2002-04-04 06:00:32, 2002-04-04 17:59:28, 200...","[2002-04-04 08:59:44, 2002-04-04 20:59:44, 200..."
4604,"[2002-03-31 15:00:16, 2002-03-31 17:59:28, 200...","[[[102549.04, 102471.04, 102387.04, 102296.04,...","[2002-03-31 04:00:00, 2002-03-31 16:00:00, 200...","[0.04060936, 0.2048425, 0.28863493, 0.2014908,...","[2002-03-31 04:59:44, 2002-03-31 16:59:44, 200...","[0.3233174, 0.54963934, 0.7172853, 0.4909633, ...","[2002-04-05 07:00:16, 2002-04-05 19:00:16, 200...","[2002-04-05 09:59:28, 2002-04-05 23:00:16, 200..."
...,...,...,...,...,...,...,...,...
5595,"[2010-10-19 08:59:44, 2010-10-19 11:58:56, 201...","[[[101470.0, 101486.0, 101501.0, 101517.0, 101...","[2010-10-18 23:00:16, 2010-10-19 10:59:12, 201...","[1.9376696, 1.6092033, 0.86177504, 1.2974956, ...","[2010-10-19 00:59:44, 2010-10-19 13:00:48, 201...","[0.021554768, 0.62508005, 0.39037576, 0.457434...","[2010-10-24 01:59:28, 2010-10-24 15:00:16, 201...","[2010-10-24 03:58:56, 2010-10-24 16:00:00, 201..."
5596,"[2010-10-20 08:59:44, 2010-10-20 12:01:04, 201...","[[[101663.87, 101650.87, 101641.87, 101635.87,...","[2010-10-20 00:00:00, 2010-10-20 12:01:04, 201...","[0.86177504, 1.2974956, 0.50984687, 0.87518185...","[2010-10-20 01:59:28, 2010-10-20 14:00:32, 201...","[0.39037576, 0.45743412, 0.28140593, 0.4322872...","[2010-10-25 01:59:28, 2010-10-25 15:00:16, 201...","[2010-10-25 04:01:04, 2010-10-25 16:00:00, 201..."
5597,"[2010-10-21 08:59:44, 2010-10-21 11:58:56, 201...","[[[101876.15, 101838.15, 101801.15, 101765.15,...","[2010-10-21 00:00:00, 2010-10-21 13:00:48, 201...","[0.50984687, 0.87518185, 1.8505255, 1.193593, ...","[2010-10-21 01:59:28, 2010-10-21 14:00:32, 201...","[0.28140593, 0.43228725, 0.15567149, 0.2478767...","[2010-10-26 02:59:12, 2010-10-26 16:00:00, 201...","[2010-10-26 05:00:48, 2010-10-26 16:59:44, 201..."


In [ ]:
Y_train

,surge1_t0,surge1_t1,surge1_t2,surge1_t3,surge1_t4,surge1_t5,surge1_t6,surge1_t7,surge1_t8,surge1_t9,surge2_t0,surge2_t1,surge2_t2,surge2_t3,surge2_t4,surge2_t5,surge2_t6,surge2_t7,surge2_t8,surge2_t9
id_sequence,,,,,,,,,,,,,,,,,,,,
4600,0.288635,0.201491,0.214898,0.325504,-0.180603,-0.867700,-0.720226,-0.230878,-0.354891,-1.460951,0.717285,0.490963,0.985519,1.639338,1.396251,1.052577,1.102871,0.843020,1.681249,1.957865
4601,0.214898,0.325504,-0.180603,-0.867700,-0.720226,-0.230878,-0.354891,-1.460951,-0.750391,-0.351539,0.985519,1.639338,1.396251,1.052577,1.102871,0.843020,1.681249,1.957865,2.469185,2.083599
4602,-0.180603,-0.867700,-0.720226,-0.230878,-0.354891,-1.460951,-0.750391,-0.351539,-0.395111,-0.455442,1.396251,1.052577,1.102871,0.843020,1.681249,1.957865,2.469185,2.083599,1.874042,1.521986
4603,-0.720226,-0.230878,-0.354891,-1.460951,-0.750391,-0.351539,-0.395111,-0.455442,-0.137031,0.399241,1.102871,0.843020,1.681249,1.957865,2.469185,2.083599,1.874042,1.521986,1.262134,0.859784
4604,-0.354891,-1.460951,-0.750391,-0.351539,-0.395111,-0.455442,-0.137031,0.399241,0.278580,-0.462145,1.681249,1.957865,2.469185,2.083599,1.874042,1.521986,1.262134,0.859784,0.574786,0.625080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595,1.451674,1.220407,0.556771,1.709754,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,0.759197,0.884931,0.298171,-0.397560,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700
5596,0.556771,1.709754,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,1.284089,0.070775,0.298171,-0.397560,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700,1.052577,1.220223
5597,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,1.284089,0.070775,0.888589,1.545521,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700,1.052577,1.220223,1.446545,2.670360


# Optional: EDA Visualizations

These were used on a brief [writeup](https://jstephens.io/projects/tideprediction/) after exporting to JavaScript or .gif format.

## Timestamp Graph



In [ ]:
def timestamp_graph(df_bokeh):
  rows = [100, 101, 102]
  data_sources = []

  for i, row in enumerate(rows):
      df_row = pd.DataFrame(df_bokeh.iloc[row]).T

      columns = ['t_slp', 't_surge1_input', 't_surge2_input', 't_surge1_output', 't_surge2_output']
      dfs = {}

      for column in columns:
          df = pd.DataFrame(df_row[column])
          df = df.explode(column).reset_index(drop=True)
          dfs[column] = df

      x = [15, 8, 1][i]
      x2 = x + 2
      x3 = x + 3

      source_row = ColumnDataSource(data=dict(
          x=pd.to_datetime(dfs['t_slp']['t_slp']),
          y=[x] * len(dfs['t_slp']),
          size=[10] * len(dfs['t_slp']),
          color=[colorvar] * len(dfs['t_slp'])
      ))
      source2_row = ColumnDataSource(data=dict(
          x=pd.to_datetime(dfs['t_surge1_input']['t_surge1_input']),
          y=[x2] * len(dfs['t_surge1_input']),
          size=[10] * len(dfs['t_surge1_input']),
          color=[colorvar] * len(dfs['t_surge1_input'])
      ))
      source3_row = ColumnDataSource(data=dict(
          x=pd.to_datetime(dfs['t_surge2_input']['t_surge2_input']),
          y=[x3] * len(dfs['t_surge2_input']),
          size=[10] * len(dfs['t_surge2_input']),
          color=[colorvar] * len(dfs['t_surge2_input'])
      ))
      source2_pred_row = ColumnDataSource(data=dict(
          x=pd.to_datetime(dfs['t_surge1_output']['t_surge1_output']),
          y=[x2] * len(dfs['t_surge1_output']),
          size=[10] * len(dfs['t_surge1_output']),
          color=['lightgray'] * len(dfs['t_surge1_output'])
      ))
      source3_pred_row = ColumnDataSource(data=dict(
          x=pd.to_datetime(dfs['t_surge2_output']['t_surge2_output']),
          y=[x3] * len(dfs['t_surge2_output']),
          size=[10] * len(dfs['t_surge2_output']),
          color=['lightgray'] * len(dfs['t_surge2_output'])
      ))

      data_sources.extend([source_row, source2_row, source3_row, source2_pred_row, source3_pred_row])


  p = figure(plot_width=800, plot_height=400, x_axis_label='Timestamp', y_axis_label='', toolbar_location=None, x_axis_type='datetime')

  y_ticks = [1,2.5,3,4,8,9.5,10,11,15,16.5,17,18]
  y_labels = ['Encoded Image Recorded','Row 102                                                               ','Surge Time, City 1','Surge Time, City 2',
              'Encoded Image Recorded','Row 101                                                               ','Surge Time, City 1','Surge Time, City 2',
              'Encoded Image Recorded','Row 100                                                               ','Surge Time, City 1','Surge Time, City 2']
  p.yaxis.ticker = FixedTicker(ticks=y_ticks)
  p.yaxis.major_label_overrides = {tick: label for tick, label in zip(y_ticks, y_labels)}

  p.ygrid.grid_line_color = 'white'

  sources = [
      source_row100, source2_row100, source3_row100, source2_pred_row100, source3_pred_row100,
      source_row101, source2_row101, source3_row101, source2_pred_row101, source3_pred_row101,
      source_row102, source2_row102, source3_row102, source2_pred_row102, source3_pred_row102
  ]

  for source in sources:
      p.circle('x', 'y', source=source, size='size', alpha=0.5,color='color')

  p.x_range=Range1d(start=pd.Timestamp('1950-11-06 12:30:00'), end=pd.Timestamp('1950-11-18 12:30:00'))

  p.y_range=Range1d(start=0, end=19)

  p.xaxis.major_tick_line_color = None
  p.xaxis.minor_tick_line_color = None
  p.yaxis.major_tick_line_color = None
  p.yaxis.minor_tick_line_color = None

  hover = HoverTool(
      tooltips=[
          ("Time", "@x{%F %T}")
      ],
      formatters={"@x": "datetime"},
      mode='mouse'
  )
  p.add_tools(hover)
  output_notebook()
  show(p)

In [ ]:
df_bokeh = df_X_train.copy()
timestamp_graph(df_bokeh)

NameError: ignored

The plot above represents three typical rows for the X training dataset. Each row represents 5 days but will overlap with other rows temporally. 

In [ ]:
script, div = components(p)
print(div)
print(script)

## Encoded Image GIF

In [ ]:
def gif(df_X_train):
  # Retrieve the 40 images from the row of interest
  images = df_X_train['slp'][5599]

  # Create an empty list to store the image file names
  image_files = []

  # Loop over each image and save it as a file
  for i in range(len(images)):
      # Create the file name with a 4-digit index
      filename = f'image_{i:04d}.png'
      
      # Save the image as a file
      plt.imsave(filename, images[i], cmap='gray')
      
      # Add the file name to the list
      image_files.append(filename)

  # Use imageio to create the gif
  with imageio.get_writer('encodedimage.gif', mode='I') as writer:
      # Loop over each image file and add it to the gif
      for filename in image_files:
          image = imageio.imread(filename)
          writer.append_data(image)

In [ ]:
gif(df_X_train)

# Timestamp Transformations



In [4]:
def get_nth_day(timestamps):
    first_timestamp = timestamps[0]
    year_start = datetime.datetime(first_timestamp.year, 1, 1)
    return int((first_timestamp - year_start).days + 1)

def get_last_date(timestamps):
    last_timestamp = timestamps[-1]
    year_start = datetime.datetime(last_timestamp.year, 1, 1)
    return int((last_timestamp - year_start).days + 1)

def get_first_year(timestamps):
    return timestamps[0].year

def extract_hour(timestamps):
    return np.array([pd.Timestamp(ts, unit='s').hour for ts in timestamps])

In [5]:
df_X_train['startingdate'] = df_X_train['t_slp'].apply(lambda x: get_nth_day(x))
df_X_train['lastdate'] = df_X_train['t_slp'].apply(lambda x: get_last_date(x))
df_X_train['year'] = df_X_train['t_slp'].apply(lambda x: get_first_year(x))
df_X_train['t_slp'] = df_X_train['t_slp'].apply(lambda x: extract_hour(x))
df_X_train['t_surge1_input'] = df_X_train['t_surge1_input'].apply(lambda x: extract_hour(x))
df_X_train['t_surge2_input'] = df_X_train['t_surge2_input'].apply(lambda x: extract_hour(x))
df_X_train['t_surge1_output'] = df_X_train['t_surge1_output'].apply(lambda x: extract_hour(x))
df_X_train['t_surge2_output'] = df_X_train['t_surge2_output'].apply(lambda x: extract_hour(x))

# Dynamic Time Warping 

This was initially used as a dimensionality reduction measure but discarded when it was found that Google Colab Pro could handle the original encoded image sequences. It's maintained here in the event it's helpful for later.

In [ ]:
def dtw_distance(s1, s2):
    n, m = len(s1), len(s2)
    dtw = np.zeros((n+1, m+1))
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = np.linalg.norm(s1[i-1] - s2[j-1])
            dtw[i][j] = cost + min(dtw[i-1][j], dtw[i][j-1], dtw[i-1][j-1])
    return dtw[n][m]

def find_closest_sequences(df, idx, k=None):
    # Extract the sequence to compare
    seq = df.iloc[idx]["slp"]
    
    # Compute DTW distance between the sequence and all prior sequences in the DataFrame
    distances = []
    for i in range(idx):
        dist = dtw_distance(seq, df.iloc[i]["slp"])
        distances.append(dist)
    distances = np.array(distances)
    
    # Get the top k indices of most similar sequences
    if k is None:
        k = len(distances)
    else:
        k = min(k, len(distances))
    top_k_indices = np.argsort(distances)[:k]
    
    # Return the closest sequences as a list of tuples (index, distance)
    closest_sequences = [(i, distances[i]) for i in top_k_indices]
    return closest_sequences

dtw_performed = False

In [ ]:
matches_list = []

for y in range(5000,5599):
  matches = find_closest_sequences(df_X_train,y,k=5599)
  matches_list.append(matches)

In [ ]:
lowest_value = 999999999999999999999999
highest_value = 0

for item in matches_list:
    if isinstance(item, list):  # Check if it's a sublist
        for pair in item:
            if pair[1] < lowest_value:
                lowest_value = pair[1]
            if pair[1] > highest_value:
                highest_value = pair[1]
    else:  # It's a tuple
        if item[1]#  < lowest_value:
            lowest_value = item[1]
        if item[1] > highest_value:
            highest_value = item[1]

dtw_performed = True

Lowest value: 83805.66455078125
Highest value: 5821887.0


# Fourier Transforms


This function applies the Fourier Transform to sequences - in this case, the 40 images seen in any given row.

In [7]:
def apply_fourier_transform(image_sequence):
    sequence_transform = np.fft.fft2(image_sequence)
    shifted_transform = np.fft.fftshift(sequence_transform, axes=(1, 2))
    magnitude_spectrum = np.abs(shifted_transform)
    return magnitude_spectrum

# Iterate over the rows of your dataframe and calculate magnitude spectra
df_X_train['magnitude_spectra'] = df_X_train['slp'].apply(apply_fourier_transform)

This function applies the Fourier Transform to individual images.

In [6]:
def apply_fourier_transform(image_sequence):
    sequence_transform = np.fft.fft2(image_sequence)
    shifted_transform = np.fft.fftshift(sequence_transform, axes=(1, 2))
    magnitude_spectrum = np.abs(shifted_transform)
    return magnitude_spectrum

# Iterate over the rows of your dataframe and calculate magnitude spectra
df_X_train['magnitude_spectra'] = df_X_train['slp'].apply(apply_fourier_transform)

Unused but kept here for posterity.

In [ ]:
def calculate_pressure_gradients(images):
    gradients = []

    for image in images:
        gradient_x = np.gradient(image, axis=1)
        gradient_y = np.gradient(image, axis=0)
        gradient_magnitude = np.sqrt(gradient_x ** 2 + gradient_y ** 2)
        gradients.append(gradient_magnitude)

    return gradients

# Assuming your dataframe is named df and the column with SLP images is named 'slp'
# Create a new column 'pressure_gradients' to store the calculated gradients
df_X_train['pressure_gradients'] = df_X_train['slp'].apply(lambda x: calculate_pressure_gradients(x))


# Dataset Prep

In [8]:
def flatten_array(row):
    new_row = []
    for col in row:
        if isinstance(col, np.ndarray):
            new_row.extend(col.ravel())
        else:
            new_row.append(col)
    return pd.Series(new_row)

df_X_train['slp'] = df_X_train['slp'].apply(lambda arr: np.concatenate(arr).ravel())

In [9]:
df_X_train['magnitude_spectra'] = df_X_train['magnitude_spectra'].apply(lambda arr: np.concatenate(arr).ravel())
df_train_flat = df_X_train.apply(flatten_array, axis=1)

In [10]:
# Convert DataFrames to numpy arrays
X = df_train_flat.to_numpy()
y = Y_train.to_numpy()

In [11]:
df_train_flat.to_csv('df_train_flat.csv')

KeyboardInterrupt: ignored

In [12]:
np.save('X.npy', X)

## X-Test Dataset Prep

All the data processing work done on the X-train dataset, repeated for X-test.

In [10]:
df_X_test['t_slp'] = df_X_test['t_slp'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_test['t_surge1_input'] = df_X_test['t_surge1_input'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_test['t_surge2_input'] = df_X_test['t_surge2_input'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_test['t_surge1_output'] = df_X_test['t_surge1_output'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_test['t_surge2_output'] = df_X_test['t_surge2_output'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))

df_X_test['startingdate'] = df_X_test['t_slp'].apply(lambda x: get_nth_day(x))
df_X_test['lastdate'] = df_X_test['t_slp'].apply(lambda x: get_last_date(x))
df_X_test['year'] = df_X_test['t_slp'].apply(lambda x: get_first_year(x))
df_X_test['t_slp'] = df_X_test['t_slp'].apply(lambda x: extract_hour(x))
df_X_test['t_surge1_input'] = df_X_test['t_surge1_input'].apply(lambda x: extract_hour(x))
df_X_test['t_surge2_input'] = df_X_test['t_surge2_input'].apply(lambda x: extract_hour(x))
df_X_test['t_surge1_output'] = df_X_test['t_surge1_output'].apply(lambda x: extract_hour(x))
df_X_test['t_surge2_output'] = df_X_test['t_surge2_output'].apply(lambda x: extract_hour(x))

if 'magnitude_spectra' in df_X_train.columns:
  df_X_test['magnitude_spectra'] = df_X_test['slp'].apply(apply_fourier_transform)
  df_X_test['magnitude_spectra'] = df_X_test['magnitude_spectra'].apply(lambda arr: np.concatenate(arr).ravel())

df_X_test['slp'] = df_X_test['slp'].apply(lambda arr: np.concatenate(arr).ravel())

In [11]:
df_test_flat = df_X_test.apply(flatten_array, axis=1)

In [12]:
X_test = df_test_flat.to_numpy()

# Model

In [12]:
regressor = xgb.XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist', gpu_id=0)
multioutput_regressor = MultiOutputRegressor(regressor).fit(X, y)

In [ ]:
# Define the parameter grid for the grid search
param_grid = {
    'estimator__learning_rate': [0.1, 0.2],
    'estimator__max_depth': [3, 5],
    'estimator__n_estimators': [100, 200]
}

# Create the XGBoost regressor with GPU settings
regressor = xgb.XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist', gpu_id=0)

# Create the MultiOutputRegressor with XGBoost regressor
multioutput_regressor = MultiOutputRegressor(regressor)

# Create the GridSearchCV object
grid_search = GridSearchCV(multioutput_regressor, param_grid, cv=3)

# Perform the grid search
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

model_file_path = f'model_{timestamp}.pkl'

joblib.dump(multioutput_regressor, model_file_path)

In [15]:
predictions = multioutput_regressor.predict(X_test)

In [ ]:
xgb_regressor = multioutput_regressor.regressor_

# Get model information
model_info = xgb_regressor.get_booster().get_dump()

# Print the model information
for tree_idx, tree_info in enumerate(model_info):
    print(f"Tree {tree_idx}:\n{tree_info}")

# Y Testing Processing and File Save

In [16]:
def fileprocessing(preds):
  timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
  preds_file_path = f'output_{timestamp}.csv'
  output_df = pd.DataFrame(preds)
  output_df.index = df_test_flat.index
  output_df.columns = Y_train.columns
  output_df.to_csv(preds_file_path)

In [17]:
fileprocessing(predictions)

# Feature Importances

In [ ]:
len(df_train_flat.columns)

In [ ]:
pip install shap

In [ ]:
pip install cudf

In [ ]:
import cudf
import shap

X_cudf = cudf.DataFrame.from_pandas(X)
explainer = shap.Explainer(multioutput_regressor.predict, X_cudf, algorithm='permutation', max_evals=2 * X.shape[1] + 1)
shap_values = explainer(X_cudf)

In [ ]:
!nvcc -V && which nvcc

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
/usr/local/cuda/bin/nvcc


In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

In [ ]:
!pip install cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!rm -rf /usr/local/lib/python3.8/dist-packages/cupy*
!pip install cupy-cuda11x

In [ ]:
!pip uninstall pyarrow
!pip install pyarrow

In [ ]:
import cudf 

In [ ]:
pip install cudf

In [ ]:
import shap
explainer = shap.Explainer(multioutput_regressor.predict, X, algorithm='permutation', max_evals=2 * X.shape[1] + 1)
shap_values = explainer(X)

In [ ]:
import cupy

In [ ]:
import shap
import cuml
X_cudf = cuml.DataFrame.from_pandas(X)

# Convert the trained model to a GPU-accelerated model
multioutput_regressor_gpu = cuml.MultiOutputRegressor(multioutput_regressor.estimator)

# Initialize the GPU-accelerated SHAP explainer
explainer = shap.Explainer(multioutput_regressor_gpu.predict, X_cudf)

# Calculate SHAP values on GPU
shap_values = explainer(X_cudf)

## Visualization: SHAP Values

# Performance Notes

Colab Pro is invaluable on a dataset of this size.


When using this implementation:

```
regressor = xgb.XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist', gpu_id=0)
multioutput_regressor = MultiOutputRegressor(regressor).fit(X, y)
```

**230 seconds** | When using 'gpu_hist' and 1000 row subset of data

**675 seconds** | When using 'gpu_hist' and entire data set


**Stopped at 7,402 seconds** | When using standard runtime and 1000 row subset  of data

**Session crashes** | When using standard runtime and entire data set


Parallel trees not used in this example but could potentially quicken the model training



